In [1]:
import numpy as np 
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import plotly.express as px
from sklearn.metrics.pairwise import cosine_similarity
import scipy.sparse
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt



In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/foodcom-recipes-and-reviews/recipes.parquet
/kaggle/input/foodcom-recipes-and-reviews/reviews.parquet
/kaggle/input/foodcom-recipes-and-reviews/reviews.csv
/kaggle/input/foodcom-recipes-and-reviews/recipes.csv
/kaggle/input/amazon-fine-food-reviews/hashes.txt
/kaggle/input/amazon-fine-food-reviews/Reviews.csv
/kaggle/input/amazon-fine-food-reviews/database.sqlite
/kaggle/input/indian-food-101/indian_food.csv


In [3]:
amz_food = pd.read_csv('/kaggle/input/amazon-fine-food-reviews/Reviews.csv')
# foodcom_recipes = pd.read_csv('/kaggle/input/foodcom-recipes-and-reviews/recipes.csv')
# indian_food = pd.read_csv('/kaggle/input/indian-food-101/indian_food.csv')

In [4]:
amz_food.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [5]:
amz_food.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Id                      568454 non-null  int64 
 1   ProductId               568454 non-null  object
 2   UserId                  568454 non-null  object
 3   ProfileName             568428 non-null  object
 4   HelpfulnessNumerator    568454 non-null  int64 
 5   HelpfulnessDenominator  568454 non-null  int64 
 6   Score                   568454 non-null  int64 
 7   Time                    568454 non-null  int64 
 8   Summary                 568427 non-null  object
 9   Text                    568454 non-null  object
dtypes: int64(5), object(5)
memory usage: 43.4+ MB


In [6]:
amz_food.shape

(568454, 10)

In [7]:
missing_val_percent = (amz_food.isna().sum()/len(amz_food)) *100
missing_val_percent

Id                        0.000000
ProductId                 0.000000
UserId                    0.000000
ProfileName               0.004574
HelpfulnessNumerator      0.000000
HelpfulnessDenominator    0.000000
Score                     0.000000
Time                      0.000000
Summary                   0.004750
Text                      0.000000
dtype: float64

In [8]:
amz_food.dtypes

Id                         int64
ProductId                 object
UserId                    object
ProfileName               object
HelpfulnessNumerator       int64
HelpfulnessDenominator     int64
Score                      int64
Time                       int64
Summary                   object
Text                      object
dtype: object

In [9]:
# 
new_df = amz_food.drop(['Id','ProfileName','HelpfulnessNumerator','HelpfulnessDenominator',
                  'Time','Summary','Text'],axis=1)

In [10]:
new_df.head()

,ProductId,UserId,Score
0,B001E4KFG0,A3SGXH7AUHU8GW,5
1,B00813GRG4,A1D87F6ZCVE5NK,1
2,B000LQOCH0,ABXLMWJIXXAIN,4
3,B000UA0QIQ,A395BORC6FGVXV,2
4,B006K2ZZ7K,A1UQRSCLF8GW1T,5


In [11]:
new_df.shape

(568454, 3)

In [12]:
new_df.isna().sum()

ProductId    0
UserId       0
Score        0
dtype: int64

In [13]:
new_df[['Score']].describe().T

,count,mean,std,min,25%,50%,75%,max
Score,568454.0,4.183199,1.310436,1.0,4.0,5.0,5.0,5.0


In [14]:
fig =px.histogram(new_df,x='Score')
fig.show()

In [15]:
# Number of unique values in UserId,ProductId
print(new_df.UserId.nunique())
print(new_df.ProductId.nunique())

256059
74258


In [16]:
top_10_users =new_df.groupby('UserId').size().sort_values(ascending=False).head(10)
top_10_users

UserId
A3OXHLG6DIBRW8    448
A1YUL9PCJR3JTY    421
AY12DBB0U420B     389
A281NPSIMI1C2R    365
A1Z54EM24Y40LL    256
A1TMAVN4CEM8U8    204
A2MUGFV2TDQ47K    201
A3TVZM3ZIXG8YW    199
A3PJZ8TU8FDQ1K    178
AQQLWCMRNDFGI     176
dtype: int64

In [17]:
counts = new_df['UserId'].value_counts()
df_final = new_df[new_df['UserId'].isin(counts[counts >= 50].index)]
df_final.head()

,ProductId,UserId,Score
14,B001GVISJM,A2MUGFV2TDQ47K,5
44,B001EO5QW8,A2G7B7FKP2O2PU,5
46,B001EO5QW8,AQLL2R1PPR46X,5
109,B001REEG6C,AY12DBB0U420B,5
141,B001GVISJW,A2YIO225BTKVPU,4


In [18]:
print(df_final.shape)
print(f"Unique Users {df_final.UserId.nunique()}")
print(f"Unique Items {df_final.ProductId.nunique()}")

(22941, 3)
Unique Users 267
Unique Items 11313


In [19]:
final_rating_matrix = pd.pivot_table(df_final,index='UserId',columns='ProductId',values='Score')
final_rating_matrix = final_rating_matrix.fillna(0)
print(f"Shape of final raitng matrix: {final_rating_matrix.shape}")
given_num_of_rating =  np.count_nonzero(final_rating_matrix)
print("given_num_of_rating ",given_num_of_rating)
possible_num_of_ratings = final_rating_matrix.shape[0] * final_rating_matrix.shape[1]
print('possible_num_of_ratings ',possible_num_of_ratings)
density = (given_num_of_rating/possible_num_of_ratings)*100
print(f"Density : {np.round(density,5)}")

Shape of final raitng matrix: (267, 11313)
given_num_of_rating  20829
possible_num_of_ratings  3020571
Density : 0.68957


In [20]:
# final_rating_matrix

In [21]:
final_rating_matrix_T = final_rating_matrix.T
final_rating_matrix_T.head()

UserId,A100WO06OQR8BQ,A106ZCP7RSXMRU,A1080SE9X3ECK0,A10G136JEISLVR,A11ED8O95W2103,A11OTLEDSW8ZXD,A12ENBT314RFXR,A132ETQPMHQ585,A13GZCHAMKWYBT,A13HRSMJ5TOWEZ,...,ATJN4KEHJBOC8,AVFA1JB08RG8G,AWKZAUC0D8DYL,AXO4PQU0XG3TG,AY12DBB0U420B,AY1EF0GOH80EK,AYB4ELCS5AM8P,AYGJ96W5KQMUJ,AYOMAHLWRQHUG,AZV26LP92E6WU
ProductId,,,,,,,,,,,,,,,,,,,,,
7310172001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7310172101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7800648702,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00004CI84,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
B00004CXX9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0


In [22]:
train_data ,test_data = train_test_split(df_final,test_size=0.3,random_state=0)
train_data.head()

,ProductId,UserId,Score
399863,B002IEVJRY,A1N5FSCYN4796F,3
20262,B001BDDTB2,A1Q7A78VSQ5GQ4,5
139611,B001BCXTGS,A2PNOU7NXB1JE4,3
455504,B005HG9ERW,A2SZLNSI5KOQJT,3
512008,B0028PDER6,ALSAOZ1V546VT,5


In [23]:
# Popularity model recommendation
train_data_grouped = train_data.groupby('ProductId')['UserId'].count().reset_index()
train_data_grouped = train_data_grouped.rename(columns = {'UserId':'score'})
train_data_grouped.head()

,ProductId,score
0,7310172001,5
1,7310172101,5
2,7800648702,1
3,B00004CI84,2
4,B00004CXX9,3


In [24]:
# train_data_grouped
train_data_sort = train_data_grouped.sort_values(['score', 'ProductId'], ascending = [0,1]) 
train_data_sort['Rank'] = train_data_sort['score'].rank(ascending=0, method='first') 
popularity_recommendations = train_data_sort.head(5) 
popularity_recommendations 

,ProductId,score,Rank
5621,B002IEZJMA,48,1.0
8130,B006MONQMC,42,2.0
5620,B002IEVJRY,41,3.0
6779,B0041NYV8E,39,4.0
7876,B005HG9ET0,39,5.0


In [25]:
# prediction for popularity based model
def recommend(user_id):
    user_recommendations = popularity_recommendations
    user_recommendations['UserId'] = user_id
    cols = user_recommendations.columns.tolist()
    cols = cols[-1:] + cols[:-1]
#     print(cols)
    user_recommendations = user_recommendations[cols]
    return user_recommendations


In [26]:
# df_final
find_recom = [44,46,109]
for i in find_recom:
    print(f" for {i}:{recommend(i)}")
    print('\n')

 for 44:      UserId   ProductId  score  Rank
5621      44  B002IEZJMA     48   1.0
8130      44  B006MONQMC     42   2.0
5620      44  B002IEVJRY     41   3.0
6779      44  B0041NYV8E     39   4.0
7876      44  B005HG9ET0     39   5.0


 for 46:      UserId   ProductId  score  Rank
5621      46  B002IEZJMA     48   1.0
8130      46  B006MONQMC     42   2.0
5620      46  B002IEVJRY     41   3.0
6779      46  B0041NYV8E     39   4.0
7876      46  B005HG9ET0     39   5.0


 for 109:      UserId   ProductId  score  Rank
5621     109  B002IEZJMA     48   1.0
8130     109  B006MONQMC     42   2.0
5620     109  B002IEVJRY     41   3.0
6779     109  B0041NYV8E     39   4.0
7876     109  B005HG9ET0     39   5.0




/tmp/ipykernel_33/4025447656.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_33/4025447656.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_33/4025447656.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Since this is a popularity-based recommender model, recommendations remain the same for all users

We predict the products based on the popularity. It is not personalized to particular user

In [27]:
# df_final

df_cf = pd.concat([train_data,test_data]).reset_index()
df_cf.head()

,index,ProductId,UserId,Score
0,399863,B002IEVJRY,A1N5FSCYN4796F,3
1,20262,B001BDDTB2,A1Q7A78VSQ5GQ4,5
2,139611,B001BCXTGS,A2PNOU7NXB1JE4,3
3,455504,B005HG9ERW,A2SZLNSI5KOQJT,3
4,512008,B0028PDER6,ALSAOZ1V546VT,5


In [28]:
pivot_df = pd.pivot_table(df_cf,index=['UserId'],columns='ProductId',values='Score')
pivot_df = pivot_df.fillna(0)
pivot_df.head()

ProductId,7310172001,7310172101,7800648702,B00004CI84,B00004CXX9,B00004RBDU,B00004RBDZ,B00004RYGX,B00004S1C6,B000052Y74,...,B009KAQZ9G,B009KAQZIM,B009KOHGEK,B009KP6HBM,B009LRLB6U,B009LT26BC,B009M2LUEW,B009PCDDO4,B009QEBGIQ,B009RB4GO4
UserId,,,,,,,,,,,,,,,,,,,,,
A100WO06OQR8BQ,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A106ZCP7RSXMRU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A1080SE9X3ECK0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A10G136JEISLVR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A11ED8O95W2103,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
pivot_df['user_index'] = np.arange(0,pivot_df.shape[0],1)
pivot_df.head()

ProductId,7310172001,7310172101,7800648702,B00004CI84,B00004CXX9,B00004RBDU,B00004RBDZ,B00004RYGX,B00004S1C6,B000052Y74,...,B009KAQZIM,B009KOHGEK,B009KP6HBM,B009LRLB6U,B009LT26BC,B009M2LUEW,B009PCDDO4,B009QEBGIQ,B009RB4GO4,user_index
UserId,,,,,,,,,,,,,,,,,,,,,
A100WO06OQR8BQ,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
A106ZCP7RSXMRU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
A1080SE9X3ECK0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
A10G136JEISLVR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
A11ED8O95W2103,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4


In [30]:
pivot_df = pivot_df.set_index(['user_index'])
pivot_df.head()

ProductId,7310172001,7310172101,7800648702,B00004CI84,B00004CXX9,B00004RBDU,B00004RBDZ,B00004RYGX,B00004S1C6,B000052Y74,...,B009KAQZ9G,B009KAQZIM,B009KOHGEK,B009KP6HBM,B009LRLB6U,B009LT26BC,B009M2LUEW,B009PCDDO4,B009QEBGIQ,B009RB4GO4
user_index,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
from scipy.sparse.linalg import svds
pivot_sparse = csr_matrix(pivot_df.values)
U , sigma , Vt = svds(pivot_sparse,k=50)
sigma = np.diag(sigma)

In [32]:
all_user_predicted_ratings = np.dot(np.dot(U,sigma),Vt)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = pivot_df.columns)
preds_df.head()

ProductId,7310172001,7310172101,7800648702,B00004CI84,B00004CXX9,B00004RBDU,B00004RBDZ,B00004RYGX,B00004S1C6,B000052Y74,...,B009KAQZ9G,B009KAQZIM,B009KOHGEK,B009KP6HBM,B009LRLB6U,B009LT26BC,B009M2LUEW,B009PCDDO4,B009QEBGIQ,B009RB4GO4
0,-0.023781,-0.023781,-0.002054,0.104898,0.104898,0.024303,0.107537,0.104898,-0.020275,-0.059207,...,0.046674,0.046674,0.016852,0.046674,-0.017740,0.097006,0.083491,0.076254,-0.057970,0.057460
1,-0.007905,-0.007905,-0.003851,-0.008111,-0.008111,-0.000537,-0.010274,-0.008111,-0.000594,0.032172,...,-0.028196,-0.028196,0.040836,-0.028196,0.002385,0.027653,0.566969,-0.027474,0.009181,0.000949
2,0.002045,0.002045,0.021680,0.053874,0.053874,-0.005837,-0.008159,0.053874,-0.046556,0.006265,...,-0.072355,-0.072355,0.038650,-0.072355,-0.047259,0.014912,-0.038054,0.122540,0.068785,0.021741
3,0.000029,0.000029,-0.000028,0.000039,0.000039,-0.000002,-0.000218,0.000039,0.000419,0.000130,...,0.000046,0.000046,0.000019,0.000046,0.000292,-0.000033,0.000224,-0.000047,0.000142,0.000023
4,0.006935,0.006935,-0.000392,0.008952,0.008952,-0.000043,0.012956,0.008952,-0.007402,0.004274,...,0.014244,0.014244,0.011886,0.014244,-0.000456,0.006861,0.045682,-0.000051,-0.007725,-0.015513


In [33]:
def recommend_items(userID,pivot_df,preds_df,num_recommendations):
    user_idx = userID -1
    
    sorted_user_rating = pivot_df.iloc[user_idx].sort_values(ascending=False)
    sorted_user_predictions = preds_df.iloc[user_idx].sort_values(ascending=False)
    
    temp = pd.concat([sorted_user_rating,sorted_user_predictions],axis=1)
    temp.index.name= "Recommended Items"
    temp.columns = ['user_ratings','user_predictions']
    temp = temp.loc[temp.user_ratings == 0]
    temp = temp.sort_values('user_predictions', ascending=False)
    print(temp.head(num_recommendations))

In [34]:
userID = 121
num_recommendations = 5
recommend_items(userID, pivot_df, preds_df, num_recommendations)

                   user_ratings  user_predictions
Recommended Items                                
B004E4EBMG                  0.0          1.553272
B004JGQ15E                  0.0          0.972833
B0061IUIDY                  0.0          0.923977
B0041NYV8E                  0.0          0.901132
B001LG940E                  0.0          0.893659


In [35]:
final_rating_matrix.head()

ProductId,7310172001,7310172101,7800648702,B00004CI84,B00004CXX9,B00004RBDU,B00004RBDZ,B00004RYGX,B00004S1C6,B000052Y74,...,B009KAQZ9G,B009KAQZIM,B009KOHGEK,B009KP6HBM,B009LRLB6U,B009LT26BC,B009M2LUEW,B009PCDDO4,B009QEBGIQ,B009RB4GO4
UserId,,,,,,,,,,,,,,,,,,,,,
A100WO06OQR8BQ,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A106ZCP7RSXMRU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A1080SE9X3ECK0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A10G136JEISLVR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A11ED8O95W2103,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
final_rating_matrix.mean().head()

ProductId
7310172001    0.037453
7310172101    0.037453
7800648702    0.018727
B00004CI84    0.044944
B00004CXX9    0.044944
dtype: float64

In [37]:
preds_df.mean().head()

ProductId
7310172001    0.001174
7310172101    0.001174
7800648702    0.004557
B00004CI84    0.039487
B00004CXX9    0.039487
dtype: float64

In [38]:
rmse_df = pd.concat([final_rating_matrix.mean(),preds_df.mean()],axis=1)
rmse_df.columns = ['Avg_actual_rating','Avg_predicted_rating']
print(rmse_df.shape)
rmse_df['item_index'] = np.arange(0, rmse_df.shape[0], 1)
rmse_df.head()

(11313, 2)


,Avg_actual_rating,Avg_predicted_rating,item_index
ProductId,,,
7310172001,0.037453,0.001174,0
7310172101,0.037453,0.001174,1
7800648702,0.018727,0.004557,2
B00004CI84,0.044944,0.039487,3
B00004CXX9,0.044944,0.039487,4


In [39]:
RMSE = round((((rmse_df.Avg_actual_rating - rmse_df.Avg_predicted_rating) ** 2).mean() ** 0.5), 5)
print(f"RMSE : {RMSE}")

RMSE : 0.00995


In [40]:
userID = 201
num_recommendations = 5
recommend_items(userID, pivot_df, preds_df, num_recommendations)

                   user_ratings  user_predictions
Recommended Items                                
B001LGGH54                  0.0          1.836298
B004JGQ15E                  0.0          1.706069
B000FFIL92                  0.0          1.554236
B007RTR9DS                  0.0          1.547637
B004E4HUMY                  0.0          1.337437


In [42]:
userID = 44
num_recommendations = 5
recommend_items(userID, pivot_df, preds_df, num_recommendations)

                   user_ratings  user_predictions
Recommended Items                                
B002IEZJMA                  0.0          2.612109
B002LANN56                  0.0          2.283994
B001OCKIP0                  0.0          1.885266
B0041CIP3M                  0.0          1.862237
B001VJ0B0I                  0.0          1.675260


In [43]:
userID = 109
num_recommendations = 5
recommend_items(userID, pivot_df, preds_df, num_recommendations)

                   user_ratings  user_predictions
Recommended Items                                
B006Q7YFLQ                  0.0          2.365940
B006Q7YG56                  0.0          2.365940
B006Q7YG24                  0.0          2.365940
B006Q7YG2O                  0.0          2.365940
B0040K41MY                  0.0          1.814241


reference : [https://www.kaggle.com/code/saurav9786/recommendation-based-on-amazon-food-review](http://)